# Tarea5: Series de Taylor

Fecha de envío del PR inicial: **viernes 21 de abril**

Fecha de aceptación del PR: **martes 25 de abril, antes de la clase**

---

## Ejercicio 1

Implementen una nueva estructura paramétrica (`type`) que defina el tipo `Taylor`, donde el parámetro debe ser un subtipo de `Number`. Definan métodos que implementen las operaciones aritméticas básicas (`+`, `-`, `*`, `/`) y la igualdad (`==`). Esto deberá ser incluido en un módulo.

Incluyan pruebas (en el archivo "runtests_taylor.jl") para cada uno de los métodos que implementen.


Aplicando la serie de Taylor a una función $f(x)$ obtenemos: 

\begin{eqnarray}
f(x) & = & f_{[0]} + f_{[1]} (x-x_0) + f_{[2]} (x-x_0)^2 + \dots + f_{[k]} (x-x_0)^k + \dots,  ...(1)\\
\end{eqnarray}

Donde cada $f_{[i]}$ es un coeficiente de la serie.

Si truncamos la serie anterior se tiene:

\begin{eqnarray}
f(x) & = & f_{[0]} + f_{[1]} (x-x_0) + f_{[2]} (x-x_0)^2 + \dots + f_{[k]} (x-x_0)^k  ...(2) \\\end{eqnarray}

A partir de (2) podemos representar los coeficientes en un arreglo como el siguiente:

\begin{eqnarray}
Taylor[f_{[0]},f_{[1]},f_{[2]},...f_{[k]}]  ...(3)\end{eqnarray}

En (3) tenemos una estructura que nos recuerda a la forma del dual: 

\begin{eqnarray}
Dual(f,df) ...(4)
\end{eqnarray}

,la cual, podemos representar ahora como:

\begin{eqnarray}
Taylor[f_{[0]},f_{[1]}]  ...(5)
\end{eqnarray}

En un inicio, tuvimos que asegurar que el dual cumpliera con las siguientes propiedades:
\begin{eqnarray}
\\
Dual(c,0)  ...(a)
\\
Dual(x,1)          ...(b)
\\
\end{eqnarray}

(Donde c es una constante y en b) se utiliza la función $f(x)=x$)


Aplicando estos casos a (3):
\begin{eqnarray}
\\
Taylor[c,0,0,...,0]  ...(c)
\\
Taylor[x,1,0,...,0]  ...(d)
\\
\end{eqnarray}

Notemos que en el caso de una constante la primera entrada es justo la constante y la segunda es 0 (su primera derivada). Por otro lado, para la variable x la primera entrada es x y la segunda es 1 (su primera derivada). . 

In [1]:
"""
Definición de polinomios de Taylor, donde
...
"""
type Taylor{T<:Number}
    # código: 
    cofun::Array{T,1} #Arreglo de los coeficientes de la función f.
    grado::Int64 #Grado de la función
    
    function Taylor(v::Vector{T},n::Int) 
        len = length(v) #Longitud del vector
        
        if len-1==n 
            return new{T}(v,n) #Si el grado ingresado es igual a los coeficientes del vector, 
                               #el grado y el vector se quedan igual.
            
            elseif len-1<n #Si el grado ingresado es mayor a los coeficientes del vector, 
                           #se completa con ceros.
            return new{T}(append!(v,zeros(T,n-len+1)),n)
        else               #Si el grado ingresado es menor a los coeficientes del vector, 
                           #se resta un grado.
            return new{T}(v,n-1)
        end
    end
       
end

Taylor

In [2]:
methods(Taylor)

# 1 method for generic function "(::Type)":
(::Type{T}){T}(arg) at sysimg.jl:53

In [3]:
Taylor([1,2,3,4],6)

LoadError: MethodError: no method matching Taylor{T<:Number}(::Array{Int64,1}, ::Int64)[0m
Closest candidates are:
  Taylor{T<:Number}{T}(::Any) at sysimg.jl:53[0m

In [4]:
#Para que lo anterior funcione debemos especificar el tipo
Taylor{Int64}([1,2,3,4],6)

Taylor{Int64}([1,2,3,4,0,0,0],6)

In [5]:
#Con esto evitamos escribir el tipo cada vez.
Taylor{T<:Number}(v::Array{T,1},n::Int)=Taylor{T}(v,n)

Taylor{T<:Number}

In [6]:
Taylor([0,1,2],2) #caso lenght(v)=grado

Taylor{Int64}([0,1,2],2)

In [7]:
Taylor([0,1,2],3) #caso lenght(v)<grado

Taylor{Int64}([0,1,2,0],3)

In [8]:
Taylor([0,1,2],1) #caso lenght(v)>grado

Taylor{Int64}([0,1,2],0)

In [9]:
#Notemos que no fue necesario hacer un promote
Taylor([1.0,2.3,5.9,4],5)

Taylor{Float64}([1.0,2.3,5.9,4.0,0.0,0.0],5)

In [10]:
#Ahora necesitamos un método para que la suma entre objetos de tipo taylor tenga sentido.

Taylor([1,2,3,4],3)+Taylor([9,8,7,6],3) #mismo grado
Taylor([1,2,3,4],7)+Taylor([9,8],2)     #diferente grado

LoadError: MethodError: no method matching +(::Taylor{Int64}, ::Taylor{Int64})[0m
Closest candidates are:
  +(::Any, ::Any, [1m[31m::Any[0m, [1m[31m::Any...[0m) at operators.jl:138[0m

Veamos cómo separar los elementos de un objeto tipo taylor

In [11]:
alfa = Taylor([0,1,2,3],5)

Taylor{Int64}([0,1,2,3,0,0],5)

In [12]:
alfa.cofun #Obtenemos solo el vector

6-element Array{Int64,1}:
 0
 1
 2
 3
 0
 0

In [13]:
alfa.cofun[4] # Obtenemos un elemento del vector

3

In [14]:
alfa.grado #Obtenemos solo el grado

5

In [12]:
5^0

1

# Operaciones

A continuación creamos los métodos necesarios para poder operar con los objetos tipo Taylor. La siguiente línea nos permite introducir las operaciones básicas de suma,resta,multiplicación,división y comparación.

In [15]:
import Base: +, -, *, /, ==

In [16]:
a1=Taylor([1,2,3],3);b1=Taylor([9,8,7,6],3)

Taylor{Int64}([9,8,7,6],3)

In [17]:
c1=Taylor([1,2,3,4],2);d1=Taylor([9,8],7)

Taylor{Int64}([9,8,0,0,0,0,0,0],7)

### Suma

\begin{eqnarray}
(f+g)_{[k]} & = & f_{[k]} + g_{[k]}
\end{eqnarray}

In [18]:
function tsuma(a,b)
    
#=---------------------Comparamos los grados----------------------------------------=#
    
    if a.grado==b.grado #Si los grados son iguales
        grado_principal=a.grado
        
    elseif a.grado>b.grado
        
        grado_principal=a.grado #El grado pricipal es el mayor de los grados
    else 
       
        grado_principal=b.grado #El grado pricipal es el mayor de los grados
        end  
    
#=---------------------Suma de los vectores-----------------------------------------=#  
    
    #Comparamos la longitud de los vectores
    
    la=length(a.cofun)
    lb=length(b.cofun)
    
    vsuma=[] #Creamos el vector suma inicialmente vacio.
             #Este es un Array{Any,1}. 
    
    if la==lb #Si la longitud de los vectores es la misma. 
        for i = 1:la
            vsuma=push!(vsuma,a.cofun[i]+b.cofun[i]) #Sumamos una a una las  componentes 
                                                     #y las agregamos al vector vsuma.   
        end
        
        elseif la<lb  
        
            lc=lb-la #Calculamos la diferencia de las longitudes
            ceros=zeros(lc) #Creamos un vector con ceros para igualar en tamaño
                            #al vector más chico.
        
            for i=1:length(ceros) #Agragamos los ceros al vector chico
                a.cofun=push!(a.cofun,ceros[i])
            end
            
        
            for i = 1:lb #Hacemos la  suma como antes.
            vsuma=push!(vsuma,a.cofun[i]+b.cofun[i]) 
            end       
   #-----     
        else
            lc=la-lb
            ceros=zeros(lc)
        
            for j=1:length(ceros)
                b.cofun=push!(b.cofun,ceros[j])
            end
            
        
            for i = 1:la
            vsuma=push!(vsuma,a.cofun[i]+b.cofun[i])
            end   
     #---   
    end
    fvsuma=convert(Array{Float64,1},vsuma) #vsuma es un arreglo tipo Any 
                                           #y necesitamos Int64 o Float64
    return Taylor(fvsuma,grado_principal)
    
end

tsuma (generic function with 1 method)

In [19]:
tsuma(a1,b1) #No es estable según el tipo

Taylor{Float64}([10.0,10.0,10.0,6.0],3)

In [20]:
tsuma(c1,d1)

Taylor{Float64}([10.0,10.0,3.0,4.0,0.0,0.0,0.0,0.0],7)

In [21]:
tsuma(d1,c1)

Taylor{Float64}([10.0,10.0,3.0,4.0,0.0,0.0,0.0,0.0],7)

Para tener la implementación de la suma,falta hacer que la siguiente línea tenga sentido:

In [22]:
c1+d1

LoadError: MethodError: no method matching +(::Taylor{Int64}, ::Taylor{Int64})[0m
Closest candidates are:
  +(::Any, ::Any, [1m[31m::Any[0m, [1m[31m::Any...[0m) at operators.jl:138[0m

Lo cual, logramos con lo siguiente:

In [24]:
+(f::Taylor, g::Taylor) = tsuma(f,g)

+ (generic function with 164 methods)

In [25]:
c1+d1

Taylor{Float64}([10.0,10.0,3.0,4.0,0.0,0.0,0.0,0.0],7)

### Resta


\begin{eqnarray}
(f-g)_{[k]} & = & f_{[k]} - g_{[k]}
\end{eqnarray}

In [26]:
function tresta(a,b)
    
#=---------------------Comparamos los grados----------------------------------------=#
    
    if a.grado==b.grado #Si los grados son iguales
        grado_principal=a.grado
        
    elseif a.grado>b.grado
        
        grado_principal=a.grado #El grado pricipal es el mayor de los grados
    else 
        
        grado_principal=b.grado #El grado pricipal es el mayor de los grados
        end  
    
#=---------------------Resta de los vectores-----------------------------------------=#  
    
    #Comparamos la longitud de los vectores
    
    la=length(a.cofun)
    lb=length(b.cofun)
    
    vresta=[] #Creamos el vector suma inicialmente vacio.
              #Este es un Array{Any,1}.
    
    if la==lb #Si la longitud de los vectores es la misma. 
        for i = 1:la
            vresta=push!(vresta,a.cofun[i]-b.cofun[i]) #Sumamos una a una las  componentes 
                                                     #y las agregamos al vector vsuma.   
        end
        
        elseif la<lb  
        
            lc=lb-la #Calculamos la diferencia de las longitudes
            ceros=zeros(lc) #Creamos un vector con ceros para igualar en tamaño
                            #al vector más chico.
        
            for i=1:length(ceros) #Agragamos los ceros al vector chico
                a.cofun=push!(a.cofun,ceros[i])
            end
            
        
            for i = 1:lb #Hacemos la  suma como antes.
            vresta=push!(vresta,a.cofun[i]-b.cofun[i]) 
            end       
   #-----     
        else
            lc=la-lb
            ceros=zeros(lc)
        
            for i=1:length(ceros)
                b.cofun=push!(b.cofun,ceros[i])
            end
            
        
            for i = 1:la
            vresta=push!(vresta,a.cofun[i]-b.cofun[i])
            end   
     #---   
    end
    fvresta=convert(Array{Float64,1},vresta) #vsuma es un arreglo tipo Any 
                                           #y necesitamos Int64 o Float64
    return Taylor(fvresta,grado_principal)
    
end

tresta (generic function with 1 method)

In [27]:
tresta(a1,b1)

Taylor{Float64}([-8.0,-6.0,-4.0,-6.0],3)

In [28]:
tresta(c1,b1)

Taylor{Float64}([-8.0,-6.0,-4.0,-2.0,0.0,0.0,0.0,0.0],2)

In [29]:
tresta(b1,c1)

Taylor{Float64}([8.0,6.0,4.0,2.0,0.0,0.0,0.0,0.0],2)

Resta de objetos tipo Taylor:

In [30]:
-(f::Taylor, g::Taylor) = tresta(f,g)

- (generic function with 191 methods)

In [31]:
b1-c1

Taylor{Float64}([8.0,6.0,4.0,2.0,0.0,0.0,0.0,0.0],2)

In [32]:
Taylor([1,2,3],3)-Taylor([1,2,3],3)

Taylor{Float64}([0.0,0.0,0.0,0.0],3)

### Multiplicación

\begin{eqnarray}
(f \cdot g)_{[k]} & = & \sum_{i=0}^k f_{[i]} \,g_{[k-i]}
\end{eqnarray}

Debemos notar que, dado un vector $v$, la entrada $v[0]$ no está definida, por lo que la suma no podrá iniciar en cero. Por ello cambiaremos los índices de la expresión anterior de modo tal que la suma inicie con $i=1$ 

\begin{eqnarray}
(f \cdot g)_{[k]} & = & \sum_{i=1}^k f_{[i]} \,g_{[k+1-i]}
\end{eqnarray}

In [34]:
function tmulti(a,b)
    
#=---------------------Multiplicación de los vectores-------------------------=#  

        #Obtenemos la longitud de los vectores
    
    la=length(a.cofun)
    lb=length(b.cofun)

    suma=0.0 #valor inicial de la suma

    if la==lb
        
        for i=1:la # La suma inicia con i=1
            suma=suma+a.cofun[i]*b.cofun[la+1-i]
        end
        
        elseif la<lb  
        
            lc=lb-la #Calculamos la diferencia de las longitudes
            ceros=zeros(lc) #Creamos un vector con ceros para igualar en tamaño
                            #al vector más chico.
        
            for i=1:length(ceros) #Agragamos los ceros al vector chico
                a.cofun=push!(a.cofun,ceros[i])
            end
        
            for i=1:la
            suma=suma+a.cofun[i]*b.cofun[la+1-i]
            end
        
        else
            lc=la-lb
            ceros=zeros(lc)
        
            for i=1:length(ceros)
                b.cofun=push!(b.cofun,ceros[i])
            end
        
            for i=1:la
            suma=suma+a.cofun[i]*b.cofun[la+1-i]
            end
        
    end    
    
    return suma
end  

tmulti (generic function with 1 method)

In [35]:
e1=Taylor([1,2,3,4],3)

Taylor{Int64}([1,2,3,4],3)

In [36]:
tmulti(e1,e1)

20.0

In [37]:
d1=Taylor([2,2,2],2)

Taylor{Int64}([2,2,2],2)

In [38]:
tmulti(d1,d1)

12.0

In [39]:
tmulti(e1,d1)

18.0

In [40]:
tmulti(d1,e1)

18.0

Multiplicación de objetos tipo Taylor:

In [42]:
*(f::Taylor, g::Taylor) = tmulti(f,g)

* (generic function with 150 methods)

In [43]:
Taylor([1,1,1,1],1)*Taylor([8,8,8,8],3)

32.0

In [44]:
Taylor([8,8,8,8],3)*Taylor([1,1,1,1],1)

32.0

In [45]:
Taylor([8,8,8,8],4)*Taylor([1,1,1,1],3)

24.0

In [46]:
Taylor([8,8,8,8],4)

Taylor{Int64}([8,8,8,8,0],4)

### División


\begin{eqnarray}
\Big(\frac{f}{g}\Big)_{[k]} & = & \frac{1}{g_{[0]}}
\Big( f_{[k]} - \sum_{i=0}^{k-1} \big(\frac{f}{g}\big)_{[i]} \, g_{[k-i]} \Big) 
\end{eqnarray}

Al igual que en el caso de la multiplicación,la suma no podrá iniciar en cero. Por ello cambiaremos los índices de la expresión anterior de modo tal que la suma inicie con $i=1$ 

\begin{eqnarray}
\Big(\frac{f}{g}\Big)_{[k]} & = & \frac{1}{g_{[1]}}
\Big( f_{[k]} - \sum_{i=1}^{k-1} \big(\frac{f}{g}\big)_{[i]} \, g_{[k+1-i]} \Big) 
\end{eqnarray}

In [47]:
function tdiv(a,b)
  #=---------------------División de los vectores-------------------------=#  

        #Obtenemos la longitud de los vectores
    
    la=length(a.cofun)
    lb=length(b.cofun)

    suma=0.0 #valor inicial de la suma

        for i=1:la
        
        if b.cofun[i]!=0 # Caso en que tenemos divisiones entre cero
 #--------------------------------------------------------------------   
            if la==lb 
                for i=1:la-1
                    suma=suma+(a.cofun[i]/b.cofun[i])*b.cofun[la+1-i]
                end

                td = (1/b.cofun[1])*(a.cofun[la]-suma)
                
                return td
                
            elseif la<lb  
        
                lc=lb-la #Calculamos la diferencia de las longitudes
                ceros=zeros(lc) #Creamos un vector con ceros para igualar en tamaño
                            #al vector más chico.
        
                for i=1:length(ceros) #Agragamos los ceros al vector chico
                    a.cofun=push!(a.cofun,ceros[i])
                end 
                #*************************************
                for i=1:la-1
                    suma=suma+(a.cofun[i]/b.cofun[i])*b.cofun[la+1-i]
                end

                td = (1/b.cofun[1])*(a.cofun[la]-suma)
                return td
                
            else  
                lc=la-lb
                ceros=zeros(lc)

                for i=1:length(ceros)
                    b.cofun=push!(b.cofun,ceros[i])
                end
                #*************************************
                for i=1:la-1
                    suma=suma+(a.cofun[i]/b.cofun[i])*b.cofun[la+1-i]
                end

                td = (1/b.cofun[1])*(a.cofun[la]-suma)
                
                
                return td
            end
#-----------------------------------------------------------------------
        else
            return println("división no definida")
        end   
            
        end       
                   
    
end

tdiv (generic function with 1 method)

In [48]:
m1=Taylor([1,2,3,4],3)

Taylor{Int64}([1,2,3,4],3)

In [49]:
tdiv(m1,m1)

-5.0

División de objetos tipo Taylor

In [50]:
/(f::Taylor, g::Taylor) = tdiv(f,g)

/ (generic function with 54 methods)

In [51]:
Taylor([1,2,3,4],3)/Taylor([4,3,2,1],3)

-0.5208333333333333

In [52]:
a1/b1

-0.6494708994708994

In [53]:
b1/d1

NaN

In [11]:
import Base: +, -, *, /, ==

# Aqui se implementan los métodos necesarios para cada función
+(f::Taylor, g::Taylor) = Taylor(f.cofun+g.cofun)
-(f::Taylor, g::Taylor) = Taylor(f.cofun-g.cofun)

- (generic function with 191 methods)

In [8]:
taylor(9,1)+taylor(3,1)

LoadError: UndefVarError: taylor not defined

In [ ]:
# Muestren que su código funciona con tests adecuados

using Base.Test
include("runtest_taylor.jl")

---

## Ejercicio 2

Obtengan las relaciones de recurrencia para las funciones $L(x) = \log\big(g(x)\big)$, $P_\alpha(x) = \big(g(x)\big)^\alpha$, $S(x) = \sin\big(g(x)\big)$, $C(x) = \cos\big(g(x)\big)$ usando el mismo procedimiento que arriba. Implementen métodos adecuados para estas funciones en el módulo, actuando sobre estructuras `Taylor` e incluyan pruebas necesarias en `runtest_taylor.jl`.